In [1]:
from transformers import (
   AutoConfig,
   AutoTokenizer,
   TFAutoModelForSequenceClassification,
   AdamW,
   glue_convert_examples_to_features
)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import sklearn
import os

In [2]:
# Choose model
# @markdown >The default model is <i><b>COVID-Twitter-BERT</b></i>. You can however choose <i><b>BERT Base</i></b> or <i><b>BERT Large</i></b> to compare these models to the <i><b>COVID-Twitter-BERT</i></b>. All these three models will be initiated with a random classification layer. If you go directly to the Predict-cell after having compiled the model, you will see that it still runs the predition. However the output will be random. The training steps below will finetune this for the specific task. <br /><br /> 
model_name = 'digitalepidemiologylab/covid-twitter-bert-v2' #@param ["digitalepidemiologylab/covid-twitter-bert", "bert-large-uncased", "bert-base-uncased"]

# Initialise tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)

In [3]:
# Training Paremeters
max_seq_length = 128 #@param {type: "integer"}
train_batch_size =  8 #@param {type: "integer"} 
eval_batch_size = 8 #@param {type: "integer"}
num_labels = 15

# Loading the Training dataset
t_train = pd.read_csv("..\\raw\\cluster_input.csv", names=('text', 'label', 'topic', 'topic_label'))
#t_train = t_train.sample(n=30000)

#t_train["text"] = t_train["text"].apply(remove_contractions)
#t_train["text"] = t_train["text"].apply(clean_text)
#t_train.drop_duplicates(subset=["text"], inplace=True)
#t_train.dropna(inplace=True)

#X_train = t_train["text"][:len(df)*0.64]
#X_val = t_train["text"][len(df)*0.64:len(df)*0.8]
#X_test = t_train["text"][len(df)*0.8:]

X_train, X_test, y_train, y_test = train_test_split(t_train['text'], t_train['topic_label'], test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=1) # 0.25 x 0.8 = 0.2

train_text = []
val_text = []
test_text = []

train_label = []
val_label = []
test_label = []

for row in X_train:
    train_text.append(str(row))
    
for row in X_val:
    val_text.append(str(row))
    
for row in X_test:
    test_text.append(str(row))
    
for row in y_train:
    train_label.append(int(row))

for row in y_val:
    val_label.append(int(row))
    
for row in y_test:
    test_label.append(int(row))

#train_text = tokenizer(train_text, max_length=max_seq_length, truncation=True, padding=True)
#val_text = tokenizer(val_text, max_length=max_seq_length, truncation=True, padding=True)
#test_text = tokenizer(test_text, max_length=max_seq_length, truncation=True, padding=True)

train = np.zeros([np.size(train_text), max_seq_length], dtype=int)
val = np.zeros([np.size(val_text), max_seq_length], dtype=int)

for i in range(len(train_text)):
    tokens = np.asarray(tokenizer.encode(train_text[i], max_length=max_seq_length, truncation=True, padding=True))
    for j in range(np.size(tokens)):
        train[i][j] = tokens[j]

for i in range(len(val_text)):
    tokens = np.asarray(tokenizer.encode(val_text[i], max_length=max_seq_length, truncation=True, padding=True))
    for j in range(np.size(tokens)):
        val[i][j] = tokens[j]


#a = tokenizer.encode(train_text[0], max_length=max_seq_length, truncation=True, padding=True)
#print(np.shape(a))


#train_text = np.asarray(train_text)
#val_text = np.asarray(val_text)
#test_text = np.asarray(test_text)

train_label = np.asarray(train_label)
val_label = np.asarray(val_label)
test_label = np.array(test_label)

label_mapping = {0:'Masks', 1:'Vaccine', 2:'Symptoms', 3:'Quarantine', 4:'Lockdown', 5:'Education', 6:'Treatment',
                 7:'Science', 8:'Statistics', 9:'Health', 10:'Work', 11:'Legislation', 12:'Politics', 13:'Travel', 14:'Testing'}

In [4]:
#train_t = tf.convert_to_tensor(train_text)
#val_t = tf.convert_to_tensor(val_text)
#test_t = tf.convert_to_tensor(test_text)
#train_l = tf.convert_to_tensor(train_label)
#val_l = tf.convert_to_tensor(val_label)
#test_l = tf.convert_to_tensor(test_label)
print(np.shape(train_text))

np.shape(train)
np.shape(train_label)

#print(np.shape(train[0]))
#print(np.shape(train[5]))
print(train[0])
print(train[5])
#print(train_text)

(9600,)
[  101  2142  7608 28324  2015  2070  2859  7599  2044 21887 23350  8293
 16770  1024  1013  1013  1056  1012  2522  1013 16950  4183  2860  2629
 11106 26775 16770  1024  1013  1013  1056  1012  2522  1013  1060  9331
  2290  2629  2497  2620 18037  2860   102     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
[  101 10722 17140 18372  2250  2386  2013  2474  8289  2013  2522 17258
  1011  2539  2420  2077  2010  7886  3367  5798 16770  1024  1013  1013
  1056  1012  2522  1013  1041  2620  2232  2595  4213  2050 27531

In [5]:
#@markdown >The default learning rate of 2e5 will be fine in most cases
learning_rate = 2e-5 #@param {type: "number"}

#@markdown > Typically these type of models are finetuned for 3 epochs. This can be increased for small datasets and decreased for large datasets.
num_epochs = 3 #@param {type: "integer"}

# Initialise a Model for Sequence Classification with 2 labels
config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, config=config)

# Optimizer and loss
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Metrics and callbacks no false negatives bitch
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
#metrics = [tf.keras.metrics.Recall(), tf.keras.metrics.Precision()]
#checkpoint_path = '..\\cvb\\checkpoints\\checkpoint.{epoch:02d}'
#callbacks = [tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True)]

# Compute some variables
use_percentage_of_data = 100
train_steps_per_epoch = int(np.size(train[:,0]) * (use_percentage_of_data/100) / train_batch_size)
dev_steps_per_epoch = int(np.size(val[:,0]) * (use_percentage_of_data/100) / eval_batch_size)
#train_steps_per_epoch = 100
#dev_steps_per_epoch = 100


# Compile model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Train the model
history = model.fit(train, train_label, epochs=num_epochs, steps_per_epoch=train_steps_per_epoch, validation_data=(val, val_label), validation_steps=dev_steps_per_epoch, batch_size=8)

'''
# Print some information about the training
print(f'\nThe training has finished training after {num_epochs} epochs.')
print('\nThe history contains the accuracy and loss at every epoch:')
print(json.dumps(history.history, indent=4))

print('\nThe checkpoint callback has generated a checkpoint after every epoch (loss being the training loss, val_loss is the validation loss):')
!ls -lha ./checkpoints/

print('\nWe will now save the finetuned model and the corresponding config file on your Colab disk.')
model.save_pretrained('..\\cvb\\huggingface_model\\')

print('\nTensorflow model and config-file is saved in ./huggingface_model/')
!ls -lha ./huggingface_model/
'''

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at digitalepidemiologylab/covid-twitter-bert-v2 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
1200/1200 [==============================] - 366s 281ms/step - loss: 2.7467 - accuracy: 0.0665 - val_loss: 2.7

InternalError:  'cuModuleGetFunction(&function, module, kernel_name)' failed with 'CUDA_ERROR_INVALID_HANDLE'
	 [[node tf_bert_for_sequence_classification/bert/encoder/layer_._23/output/LayerNorm/batchnorm/sub (defined at \shahzehan\miniconda3\envs\dick\lib\site-packages\transformers\models\bert\modeling_tf_bert.py:372) ]] [Op:__inference_train_function_33082]

Function call stack:
train_function


In [ ]:

# Small function only used for formatting the output
def format_prediction(preds, label_mapping, label_name):
    preds = tf.nn.softmax(preds, axis=1)
    formatted_preds = []
    for pred in preds.numpy():
        # convert to Python types and sort
        pred = {label: float(probability) for label, probability in zip(label_mapping.values(), pred)}
        pred = {k: v for k, v in sorted(pred.items(), key=lambda item: item[1], reverse=True)}
        formatted_preds.append({label_name: list(pred.keys())[0], f'{label_name}_probabilities': pred})
    return formatted_preds

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import f1_score

def evaluation_summary(description, predictions, true_labels):
    print("Evaluation for: " + description)
    precision = precision_score(predictions, true_labels, average='macro')
    recall = recall_score(predictions, true_labels, average='macro')
    accuracy = accuracy_score(predictions, true_labels)
    f1 = f1_score(predictions, true_labels, average='macro')
    #f1 = fbeta_score(predictions, true_labels, 1, average='macro') #1 means f_1 measure
    print("Classifier '%s' has Acc=%0.3f P=%0.3f R=%0.3f F1=%0.3f" % (description,accuracy,precision,recall,f1))
    print(classification_report(predictions, true_labels, digits=3, zero_division=0))
    print('\nConfusion matrix:\n',confusion_matrix(true_labels, predictions))

In [ ]:
test = np.zeros([np.size(test_text), max_seq_length], dtype=int)

for i in range(len(test_text)):
    tokens = np.asarray(tokenizer.encode(test_text[i], max_length=max_seq_length, truncation=True, padding=True))
    for j in range(np.size(tokens)):
        test[i][j] = tokens[j]

In [ ]:
test_pred = model.predict(test)

In [ ]:
formatted_test_preds = format_prediction(test_pred[0], label_mapping, 'fake?')

In [ ]:
test_pred_labels = []
for i in range(len(test_text)):
    if (formatted_test_preds[i]['fake?_probabilities']['Real'] > 0.5):
        test_pred_labels.append(0)
    else:
        test_pred_labels.append(1)

In [ ]:
evaluation_summary("Covid-Twitter-Bert-v2", test_pred_labels, test_label)